https://medium.com/@franky07724_57962/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1

In [1]:
#Importing the ResNet50 model
from keras.applications.resnet50 import ResNet50, preprocess_input

#Loading the ResNet50 model with pre-trained ImageNet weights
# include_top = False loads until the penultimate layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

Using TensorFlow backend.


94658560/94653016 [==============================] - 394s 4us/step


In [15]:
#scaler = transforms.Scale((224, 224))
#normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225])
#to_tensor = transforms.ToTensor()
import numpy as np
from PIL import Image
pic_one_path = '../Data/hymenoptera_data/train/ants/swiss-army-ant.jpg'

image = Image.open(pic_one_path)

print("Creating numpy representation of image {}".format(pic_one_path))
resize = image.resize((200,200), Image.NEAREST) 
resize.load()
data = np.asarray(resize, dtype="uint8" )
print(data.shape)

Creating numpy representation of image ../Data/hymenoptera_data/train/ants/swiss-army-ant.jpg
(200, 200, 3)


In [23]:

#Reshaping the training data
#X_train_new = np.array([imresize(X_train[i], (200, 200, 3)) for i in range(0, len(X_train))]).astype('float32')

X_train_new = np.array([data]).astype('float32')

#Preprocessing the data, so that it can be fed to the pre-trained ResNet50 model. 
resnet_train_input = preprocess_input(X_train_new)

#Creating bottleneck features for the training data
train_features = model.predict(resnet_train_input)

vector = train_features.reshape(1,2048)

#Saving the bottleneck features
np.savez('resnet_features_train', features=train_features)
vector.shape

(1, 2048)

In [22]:
print("type(train_features) = {} and train_features.shape = {}".format(type(train_features), train_features.shape))

type(train_features) = <class 'numpy.ndarray'> and train_features.shape = (1, 1, 1, 2048)


In [60]:
#Importing the ResNet50 model
from keras.applications.resnet50 import ResNet50, preprocess_input

#Loading the ResNet50 model with pre-trained ImageNet weights
# include_top = False loads until the penultimate layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

In [65]:
def get_vector(image_path):
    # 1. Load the image with Pillow library
    image = Image.open(image_path)
    
    # 2. Resize image to the shape acceptable as input to the ResNet50 model
    resized_image = image.resize((200, 200), Image.NEAREST)
    resized_image.load()
    
    # 3. Preprocess the resized image so that in can be fed as an input to ResNet50
    resized_image = np.asarray(resized_image, dtype="uint8" )
    X = np.array([resized_image]).astype('float32')
    resnet_input = preprocess_input(X)
    
    # 4. Forward pass through model
    last_layer_activations = model.predict(resnet_input)
    
    # 5. Reshape the activations to 1 dimensional vector
    vector = last_layer_activations.reshape(1, 2048)
    
    return vector

In [81]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
from sklearn.metrics.pairwise import cosine_similarity

v_cat = get_vector('/home/raghav/Practice/MLPractice/Data/cats_and_dogs/cat.jpeg')
v_dog = get_vector('../Data/cats_and_dogs/dog.jpeg')
v_another_cat = get_vector('../Data/cats_and_dogs/another_cat.jpeg')
v_another_dog = get_vector('../Data/cats_and_dogs/another_dog.jpeg')

print("similarity between cat and dog = {}".format(np.asscalar(cosine_similarity(v_cat, v_dog))))
print("similarity between cat and another_cat = {}".format(np.asscalar(cosine_similarity(v_cat, v_another_cat))))
print("similarity between dog and another_dog = {}".format(np.asscalar(cosine_similarity(v_dog, v_another_dog))))
print("similarity between another_dog and another_cat = {}".format(np.asscalar(cosine_similarity(v_another_dog, v_another_cat))))

similarity between cat and dog = 0.35899585485458374
similarity between cat and another_cat = 0.657699465751648
similarity between dog and another_dog = 0.49333345890045166
similarity between another_dog and another_cat = 0.3128197193145752
